imports

In [1]:
from tqdm.notebook import tqdm as tqdm
import os
import json
import pandas as pd

from HGNN.train.configParser import ConfigParser, getModelName, getDatasetName
from myhelpers import TrialStatistics

parameters

In [2]:
experimentsPath="/home/elhamod/HGNN/experiments/"
dataPath="/data/BGNN_data"
experimentName="MoreExperimentsFineTune"

cuda=6

file and changes

In [3]:
old_params_file = os.path.join(experimentsPath, experimentName, "params.json")

# Keep same order as ConfigParserWriter!
changes = {

}

prepare

In [4]:
config_parser = ConfigParser(experimentsPath, dataPath, experimentName)
paramsIterator = config_parser.getExperiments()  
number_of_experiments = sum(1 for e in paramsIterator)

# populate a hash table of all experiments, indexed by trial hash
experiments = {}
for i, experiment_params in enumerate(config_parser.getExperiments(fixExperiments=False)):
    num_of_models = experiment_params["numOfTrials"]
    for k in range(num_of_models):
        old_trialHash = TrialStatistics.getTrialName(experiment_params, k)
        experiment_params_new = {**experiment_params, **changes} 
        experiment_params_new = config_parser.fixExperimentParams(experiment_params_new)
        new_trialHash = TrialStatistics.getTrialName(experiment_params_new, k)
        experiments[old_trialHash] = {"trialHash": new_trialHash}

Fix data splits

In [5]:
with tqdm(total=number_of_experiments, desc="experiment") as bar:
    for i, experiment_params in enumerate(config_parser.getExperiments(fixExperiments=False)):
        
        # update experiment params
        experiment_params_new = {**experiment_params, **changes} 
        experiment_params_new = config_parser.fixExperimentParams(experiment_params_new)

        # fix data split names
        old_dataset_name = getDatasetName(config_parser.fixPaths(experiment_params))
        old_datasplitsPath = os.path.join(experimentsPath, experimentName, old_dataset_name)
        new_dataset_name = getDatasetName(config_parser.fixPaths(experiment_params_new))
        new_datasplitsPath = os.path.join(experimentsPath, experimentName, new_dataset_name)
        
        try:
            if os.path.exists(old_datasplitsPath):

                # update the folder names
                os.rename(old_datasplitsPath, new_datasplitsPath)
                print("datasplit", old_datasplitsPath, '->', new_datasplitsPath, "updated")

                # update the params.json
                j = json.dumps(experiment_params_new)
                json_fileName = os.path.join(new_datasplitsPath, 'params.json')
                f = open(json_fileName,"w")        
                f.write(j)
                f.close()  
                print("json", json_fileName, "updated")
                
            else:
                raise
                
        except:
            print("datasplit", old_dataset_name, "could not be fixed to", new_dataset_name)
            pass   
        
        experiments[TrialStatistics.getTrialName(experiment_params, 0)]['datasetName'] = new_dataset_name

        
        bar.update()

datasplit /home/elhamod/HGNN/experiments/MoreExperimentsFineTune/datasplits/8abeaa923bd09bce60acac49f8d867e75f3c57d47370f417140df18a -> /home/elhamod/HGNN/experiments/MoreExperimentsFineTune/datasplits/8abeaa923bd09bce60acac49f8d867e75f3c57d47370f417140df18a updated
json /home/elhamod/HGNN/experiments/MoreExperimentsFineTune/datasplits/8abeaa923bd09bce60acac49f8d867e75f3c57d47370f417140df18a/params.json updated
datasplit /home/elhamod/HGNN/experiments/MoreExperimentsFineTune/datasplits/8abeaa923bd09bce60acac49f8d867e75f3c57d47370f417140df18a -> /home/elhamod/HGNN/experiments/MoreExperimentsFineTune/datasplits/8abeaa923bd09bce60acac49f8d867e75f3c57d47370f417140df18a updated
json /home/elhamod/HGNN/experiments/MoreExperimentsFineTune/datasplits/8abeaa923bd09bce60acac49f8d867e75f3c57d47370f417140df18a/params.json updated
datasplit /home/elhamod/HGNN/experiments/MoreExperimentsFineTune/datasplits/8abeaa923bd09bce60acac49f8d867e75f3c57d47370f417140df18a -> /home/elhamod/HGNN/experiments/Mor

Fix models

In [6]:
with tqdm(total=number_of_experiments, desc="experiment") as bar:
    for i, experiment_params in enumerate(config_parser.getExperiments(fixExperiments=False)):
        
        experiment_params_new = {**experiment_params, **changes} 
        experiment_params_new = config_parser.fixExperimentParams(experiment_params_new)

        num_of_models = experiment_params["numOfTrials"]
        with tqdm(total=num_of_models, desc="trial") as bar2:
            for k in range(num_of_models):
            
                # Fix model names
                old_model_name = getModelName(experiment_params, k)
                old_modelPath = os.path.join(experimentsPath, experimentName, old_model_name)
                new_model_name = getModelName(experiment_params_new, k)
                new_modelPath = os.path.join(experimentsPath, experimentName, new_model_name)

                try:
                    if os.path.exists(old_modelPath):
                        os.rename(old_modelPath, new_modelPath)
                        print("model", old_modelPath, '->', new_modelPath, "updated")

                        j = json.dumps(experiment_params_new)
                        json_fileName = os.path.join(new_modelPath, 'params.json')
                        f = open(json_fileName,"w")        
                        f.write(j)
                        f.close()  
                        print("json", json_fileName, "updated")
                    else:
                        raise
                    
                except:
                    print("model", old_model_name, "could not be fixed to", new_model_name)
                    pass  
                
                

                bar2.update()
                
                experiments[TrialStatistics.getTrialName(experiment_params, k)]['modelName'] = new_model_name

        bar.update()

model /home/elhamod/HGNN/experiments/MoreExperimentsFineTune/models/63f07da9402527a19f4cb7fc81ba5ff837963f98a89642a85a0e4115 -> /home/elhamod/HGNN/experiments/MoreExperimentsFineTune/models/e54637afeabef3ba1bfda6ff344ca33960ac1a7c4e8b97bd708f17a3 updated
json /home/elhamod/HGNN/experiments/MoreExperimentsFineTune/models/e54637afeabef3ba1bfda6ff344ca33960ac1a7c4e8b97bd708f17a3/params.json updated
model /home/elhamod/HGNN/experiments/MoreExperimentsFineTune/models/1511d0ea1ff040251d83d79fd09d95fee8b5fd26a79b9db189bdcca1 -> /home/elhamod/HGNN/experiments/MoreExperimentsFineTune/models/aec6f2fc19e1131bf3967eab0e04a666fcd0b2cf40c7366ce06210e1 updated
json /home/elhamod/HGNN/experiments/MoreExperimentsFineTune/models/aec6f2fc19e1131bf3967eab0e04a666fcd0b2cf40c7366ce06210e1/params.json updated
model /home/elhamod/HGNN/experiments/MoreExperimentsFineTune/models/e6d0cf6718b90964d51cc43c432d60b55a1a276c30ebcd583b59a2ad -> /home/elhamod/HGNN/experiments/MoreExperimentsFineTune/models/a72d57c5006e

model /home/elhamod/HGNN/experiments/MoreExperimentsFineTune/models/70e1140e973a6d54d0182ee94ceb19f011de7a919cf1e36be8e72bab -> /home/elhamod/HGNN/experiments/MoreExperimentsFineTune/models/2a0cb954e72e6052b04d0d64a5cb68f395b0c81b8490cb6956497835 updated
json /home/elhamod/HGNN/experiments/MoreExperimentsFineTune/models/2a0cb954e72e6052b04d0d64a5cb68f395b0c81b8490cb6956497835/params.json updated
model /home/elhamod/HGNN/experiments/MoreExperimentsFineTune/models/69d236330f76c1e00f43f67b7de3b9a0fdc3c7b92d1b6f95995f3ef2 -> /home/elhamod/HGNN/experiments/MoreExperimentsFineTune/models/0f26619e3a8960853124d2a3ac19c2735b69a1ed273ede5f34ff386f updated
json /home/elhamod/HGNN/experiments/MoreExperimentsFineTune/models/0f26619e3a8960853124d2a3ac19c2735b69a1ed273ede5f34ff386f/params.json updated
model /home/elhamod/HGNN/experiments/MoreExperimentsFineTune/models/cf69d9a84d0a0370d01e8d8e951fa44a7cae1ac83f57356a083cdbc2 -> /home/elhamod/HGNN/experiments/MoreExperimentsFineTune/models/818fa776e0fd

model /home/elhamod/HGNN/experiments/MoreExperimentsFineTune/models/5e98e85ac8b4d46b2bb165519929c3ec9f5b20c407e8fbb0f774c360 -> /home/elhamod/HGNN/experiments/MoreExperimentsFineTune/models/c6bd01dd65f23bf8b19142a49d8cc0de5455ee8108c3c83229053f02 updated
json /home/elhamod/HGNN/experiments/MoreExperimentsFineTune/models/c6bd01dd65f23bf8b19142a49d8cc0de5455ee8108c3c83229053f02/params.json updated
model /home/elhamod/HGNN/experiments/MoreExperimentsFineTune/models/a07151aed1ff774feb74f409b7903784c05bd570eaf9d607cfd6bf38 -> /home/elhamod/HGNN/experiments/MoreExperimentsFineTune/models/bcbc7278f6b6e1d9978df8d2efdb995520874f3cb19097f2f4e5ed19 updated
json /home/elhamod/HGNN/experiments/MoreExperimentsFineTune/models/bcbc7278f6b6e1d9978df8d2efdb995520874f3cb19097f2f4e5ed19/params.json updated
model /home/elhamod/HGNN/experiments/MoreExperimentsFineTune/models/f800522885b194acc08ad89817694d1aba0770b8e0fd0b6f318defc3 -> /home/elhamod/HGNN/experiments/MoreExperimentsFineTune/models/4de15524d36d

Fix results directory

In [7]:
with tqdm(total=number_of_experiments, desc="experiment") as bar:
    for i, experiment_params in enumerate(config_parser.getExperiments(fixExperiments=False)):
        
        experiment_params_new = {**experiment_params, **changes} 
        experiment_params_new = config_parser.fixExperimentParams(experiment_params_new)

        old_experimentHash = TrialStatistics.getTrialName(experiment_params)
        old_experimentPath = os.path.join(experimentsPath, experimentName, "results", old_experimentHash)
        new_experimentHash = TrialStatistics.getTrialName(experiment_params_new)
        new_experimentPath = os.path.join(experimentsPath, experimentName, "results", new_experimentHash)
        
        try:
            if os.path.exists(old_experimentPath):

                os.rename(old_experimentPath, new_experimentPath)
                print("result", old_experimentPath, '->', new_experimentPath, "updated")

                j = json.dumps(experiment_params_new)
                json_fileName = os.path.join(new_experimentPath, 'params.json')
                f = open(json_fileName,"w")        
                f.write(j)
                f.close()  
                print("json", json_fileName, "updated")
            else:
                raise
                
        except:
            print("result", old_experimentHash, "could not be fixed to",new_experimentHash)
            pass 
        
        experiments[TrialStatistics.getTrialName(experiment_params, 0)]['experimentHash'] = new_experimentHash
        
        bar.update()

result 3a4aecb555ce59e2f8cf5fead0f8cb491647470e89f0c3503d0d8fff could not be fixed to 48b2f0930342601f1f0704b9b3f54e685da043fb5e6e374810da76d1
result 96dc11b7b5a771c5faf3f36905d39702667b8265ebc8af291cfbff50 could not be fixed to eab44f2bd5bc899e6e189a4cf8dbd2efdd911fdab94fab8cb8a68a8b
result 272d46c4c703293e460eb2dd4dda612f91ecab23f3e910e767d99268 could not be fixed to 835c0f0a7f8234e36d54619abacf1559236457cc69186e32d793cfd0



Fix experiments file

In [8]:
# open experiments.csv file
experimentsFileNameAndPath=os.path.join(experimentsPath, "experiments.csv")
experiments_file= pd.read_csv(experimentsFileNameAndPath)

# For all experiments
with tqdm(total=number_of_experiments, desc="experiment") as bar:
    for i, experiment_params in enumerate(config_parser.getExperiments(fixExperiments=False)):
        
        numOfTrials = experiment_params["numOfTrials"]
        trialHash_0 = TrialStatistics.getTrialName(experiment_params, 0) # used for keys who don't change for different trials
        experiment_params_new = {**experiment_params, **changes} 
        experiment_params_new = config_parser.fixExperimentParams(experiment_params_new)
        
        # For all trials
        with tqdm(total=num_of_models, desc="trial") as bar2:
            for j in range(numOfTrials):
                
                trialHash = TrialStatistics.getTrialName(experiment_params, j)
                try:
                    # If old entry doesn't exist, nothing to update
                    record_exists = not (experiments_file[experiments_file['trialHash'] == trialHash][experiments_file['experimentName'] == experimentName]).empty
                    if record_exists:
                        
                        # drop old row and caluclate new row
                        new_trialHash = experiments[trialHash]["trialHash"]
                        experiments_file.drop(experiments_file[experiments_file['trialHash'] == trialHash][experiments_file['experimentName'] == experimentName].index, inplace = True) 
                        row_information = {
                            'experimentName': experimentName,
                            'modelName': experiments[trialHash]["modelName"],
                            'datasetName': experiments[trialHash_0]["datasetName"],
                            'experimentHash': experiments[trialHash_0]["experimentHash"],
                            'trialHash': new_trialHash
                        }
                        row_information = {**row_information, **experiment_params_new} 

                        # error handling to avoid duplicates
                        record_exists = not (experiments_file[experiments_file['trialHash'] == new_trialHash][experiments_file['experimentName'] == experimentName]).empty
                        if record_exists:
                            experiments_file.drop(experiments_file[experiments_file['trialHash'] == new_trialHash][experiments_file['experimentName'] == experimentName].index, inplace = True) 

                        # update with new entry    
                        experiments_file = experiments_file.append(pd.DataFrame(row_information, index=[0]), ignore_index = True)
                        print("trialHash", trialHash, "->", new_trialHash, "updated")
                except:
                    print("trialHash", trialHash, "could not be updated!")
                    pass
                bar2.update()
        bar.update()

# resave file
experiments_file.to_csv(experimentsFileNameAndPath, header=True, index=False)

trialHash 63f07da9402527a19f4cb7fc81ba5ff837963f98a89642a85a0e4115 -> e54637afeabef3ba1bfda6ff344ca33960ac1a7c4e8b97bd708f17a3 updated
trialHash 1511d0ea1ff040251d83d79fd09d95fee8b5fd26a79b9db189bdcca1 -> aec6f2fc19e1131bf3967eab0e04a666fcd0b2cf40c7366ce06210e1 updated
trialHash e6d0cf6718b90964d51cc43c432d60b55a1a276c30ebcd583b59a2ad -> a72d57c5006e83ac4a445a2930361ea8b34e10d4f64b369f4b641038 updated
trialHash 3f993e3e8223dc3adc2b9c1be0d38f6a7e0497d487e29525a430120c -> e148e4dfff5386a8132fa24ac40b8a3035585a27112343526f3d618e updated
trialHash 4e018633bea985bc043cc6ba729b39272e9c24867680d228ca018074 -> cac707e326387cd0fcf04a98a39b6172a1d45fc4d6ec22cc3583b344 updated



/home/elhamod/melhamodenv3/lib/python3.6/site-packages/ipykernel_launcher.py:21: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/home/elhamod/melhamodenv3/lib/python3.6/site-packages/ipykernel_launcher.py:26: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/home/elhamod/melhamodenv3/lib/python3.6/site-packages/ipykernel_launcher.py:37: UserWarning: Boolean Series key will be reindexed to match DataFrame index.


trialHash 70e1140e973a6d54d0182ee94ceb19f011de7a919cf1e36be8e72bab -> 2a0cb954e72e6052b04d0d64a5cb68f395b0c81b8490cb6956497835 updated
trialHash 69d236330f76c1e00f43f67b7de3b9a0fdc3c7b92d1b6f95995f3ef2 -> 0f26619e3a8960853124d2a3ac19c2735b69a1ed273ede5f34ff386f updated
trialHash cf69d9a84d0a0370d01e8d8e951fa44a7cae1ac83f57356a083cdbc2 -> 818fa776e0fd137aca4ed1361781639bd2dbc2d183ed8b59ac0fc3c0 updated
trialHash 4ff879d4829fe8d5b5abc3439e381e31cfc99001f6691267cf3a0d25 -> 288ec57f5b90cfadc4caa125b8682cf3add4b84e7ca73fd821017325 updated
trialHash 402b81e89f98caa15dee087c1f42bbb30254a5e4269c81559b153b11 -> 38c8710896fb7650c429518b1e66426ea2dccb51cfb318fefb6a19f3 updated



trialHash 5e98e85ac8b4d46b2bb165519929c3ec9f5b20c407e8fbb0f774c360 -> c6bd01dd65f23bf8b19142a49d8cc0de5455ee8108c3c83229053f02 updated
trialHash a07151aed1ff774feb74f409b7903784c05bd570eaf9d607cfd6bf38 -> bcbc7278f6b6e1d9978df8d2efdb995520874f3cb19097f2f4e5ed19 updated




Fix params file

In [9]:
experiment_list = []
with tqdm(total=number_of_experiments, desc="experiment") as bar:
    for i, experiment_params in enumerate(config_parser.getExperiments(fixExperiments=False)):
        
        experiment_params_new = {**experiment_params, **changes} 
        experiment_list.append(experiment_params_new)
        bar.update()


j = json.dumps({"experimentList": experiment_list})
f = open(old_params_file,"w")        
f.write(j)
f.close()  
print("json", old_params_file, "updated")


json /home/elhamod/HGNN/experiments/MoreExperimentsFineTune/params.json updated
